# Get_Original_AlphaNet_Data

In [1]:
Alpha_Name = "AlphaNet_Original_Input_10d_return_move7"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "10d_ret"

In [2]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")

# Get_Original_AlphaNet_Data

In [ ]:
dir_list = ['Data','Cov','Corr','Return','Std','Zscore','Decay']
for i in dir_list:
    os.makedirs(path+i)

In [7]:
df = pd.read_csv(path+"data.csv")
df.dropna(inplace=True,axis=0,how="any")
df

,timestamp,ticker,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,10d_ret
0,2015-01-06 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,5.7657,-0.131419
1,2015-01-06 09:30:00,2,1774.23,1885.20,1753.27,1838.34,1823.031224,6560835.70,7.2662,6.7834,8.1741,-0.154144
2,2015-01-06 09:30:00,4,63.31,64.62,62.18,63.76,63.277989,28095.12,0.7707,3.3496,4.5343,0.020360
3,2015-01-06 09:30:00,5,38.00,38.00,38.00,38.00,37.997172,0.00,0.0000,0.0000,0.0000,0.000000
4,2015-01-06 09:30:00,6,225.64,234.85,223.10,225.00,228.438278,1078647.12,0.4255,8.0562,13.2751,-0.134449
...,...,...,...,...,...,...,...,...,...,...,...,...
5357699,2021-07-01 09:30:00,688700,47.80,54.50,47.03,53.38,50.715300,64336.83,14.5494,19.2222,19.2222,0.066883
5357700,2021-07-01 09:30:00,688777,90.21,95.08,89.32,94.42,93.250464,19001.61,4.6111,4.6288,4.6288,0.033394
5357701,2021-07-01 09:30:00,688788,138.07,149.35,137.11,146.22,145.197985,13572.56,5.1954,7.7977,7.7977,-0.136999
5357702,2021-07-01 09:30:00,688819,44.04,45.64,43.95,45.15,45.026221,47249.01,1.8544,4.6754,4.6754,-0.014273


In [9]:
# for ticker in tqdm(df['ticker'].drop_duplicates()):
def test(one_data):
    one_data.set_index(['timestamp','ticker'],inplace=True)
    target_df = one_data[target]
    one_data.drop(target,axis=1,inplace=True)
    concat_df = pd.DataFrame()
    for day in range(29,-1,-1):
        concat_df = pd.concat([concat_df,one_data.shift(day)],axis=1)
    concat_df.dropna(axis=0,how="any",inplace=True)
    concat_df = pd.concat([concat_df,target_df],axis=1,join='inner')
    return concat_df

In [10]:
if __name__=="__main__":
    pool = mp.Pool(10)
    result = {}
    t1 = time.time()
    grouped = df.groupby("ticker")
    for ticker , one_data in tqdm(grouped):
        result[ticker] = (pool.apply_async(test, args=(one_data,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
    pool.close()
    pool.join()
    t2 = time.time()
    print("并行执行时间：", int(t2 - t1))
    df_list = []
    for i in tqdm(result):
        df_list.append(result[i].get())
    final_df = pd.concat(df_list)
final_df

100%|██████████| 4427/4427 [00:00<00:00, 109325.58it/s]

并行执行时间： 46


,,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,open,...,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,10d_ret
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,5.7657,1126.20,...,989.78,1007.54,983.38,991.20,996.094771,887743.10,0.6494,0.9025,1.7894,-0.034691
2015-02-17 09:30:00,1,1126.20,1164.57,1104.89,1121.23,1132.652885,2166421.40,-1.4981,2.2024,4.3668,1105.60,...,991.20,995.46,975.57,989.07,985.109868,721202.39,-0.2151,0.7332,1.4537,-0.008664
2015-02-25 09:30:00,1,1105.60,1124.78,1087.12,1099.91,1101.168988,1700120.67,-1.9011,1.7283,3.4269,1101.33,...,991.91,1002.57,989.78,994.04,996.570831,680491.59,0.5029,0.6918,1.3717,0.007681
2015-02-26 09:30:00,1,1101.33,1106.31,1058.70,1062.96,1074.097527,1407714.21,-3.3592,1.4311,2.8375,1058.70,...,997.59,998.30,977.70,980.54,983.482739,577392.33,-1.3581,0.5870,1.1638,0.038965
2015-02-27 09:30:00,1,1058.70,1127.62,1045.20,1071.49,1086.375607,2508500.23,0.8021,2.5501,5.0564,1056.57,...,980.54,1000.44,966.33,999.73,986.644628,1025544.24,1.9565,1.0426,2.0672,0.070719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-25 09:30:00,688981,54.50,55.95,54.50,55.82,55.353800,209785.85,0.6310,1.9147,1.9147,56.70,...,58.53,59.11,57.55,57.70,58.173300,273520.23,-1.4349,2.4964,2.4964,-0.029360
2021-06-28 09:30:00,688981,56.70,56.98,55.60,55.86,56.113100,260448.82,0.0717,2.3771,2.3771,55.62,...,57.76,59.99,57.76,59.31,59.274600,474988.89,2.7903,4.3352,4.3352,-0.055569
2021-06-29 09:30:00,688981,55.62,56.40,55.05,55.08,55.686500,203770.43,-1.3963,1.8598,1.8598,55.19,...,59.85,60.19,58.86,59.06,59.476200,341515.10,-0.4215,3.1170,3.1170,-0.037105


In [11]:
final_df.columns = [str(i) for i in range(final_df.shape[1])]
final_df.rename(columns={str(final_df.shape[1] - 1):'target'}, inplace=True)
final_df.reset_index(inplace=True)
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])
final_df.dropna(axis=0,inplace=True) # delete none
final_df

,timestamp,ticker,0,1,2,3,4,5,6,7,...,261,262,263,264,265,266,267,268,269,target
0,2015-02-16 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,...,989.78,1007.54,983.38,991.20,996.094771,887743.10,0.6494,0.9025,1.7894,-0.034691
1,2015-02-17 09:30:00,1,1126.20,1164.57,1104.89,1121.23,1132.652885,2166421.40,-1.4981,2.2024,...,991.20,995.46,975.57,989.07,985.109868,721202.39,-0.2151,0.7332,1.4537,-0.008664
2,2015-02-25 09:30:00,1,1105.60,1124.78,1087.12,1099.91,1101.168988,1700120.67,-1.9011,1.7283,...,991.91,1002.57,989.78,994.04,996.570831,680491.59,0.5029,0.6918,1.3717,0.007681
3,2015-02-26 09:30:00,1,1101.33,1106.31,1058.70,1062.96,1074.097527,1407714.21,-3.3592,1.4311,...,997.59,998.30,977.70,980.54,983.482739,577392.33,-1.3581,0.5870,1.1638,0.038965
4,2015-02-27 09:30:00,1,1058.70,1127.62,1045.20,1071.49,1086.375607,2508500.23,0.8021,2.5501,...,980.54,1000.44,966.33,999.73,986.644628,1025544.24,1.9565,1.0426,2.0672,0.070719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5230320,2021-06-25 09:30:00,688981,54.50,55.95,54.50,55.82,55.353800,209785.85,0.6310,1.9147,...,58.53,59.11,57.55,57.70,58.173300,273520.23,-1.4349,2.4964,2.4964,-0.029360
5230321,2021-06-28 09:30:00,688981,56.70,56.98,55.60,55.86,56.113100,260448.82,0.0717,2.3771,...,57.76,59.99,57.76,59.31,59.274600,474988.89,2.7903,4.3352,4.3352,-0.055569
5230322,2021-06-29 09:30:00,688981,55.62,56.40,55.05,55.08,55.686500,203770.43,-1.3963,1.8598,...,59.85,60.19,58.86,59.06,59.476200,341515.10,-0.4215,3.1170,3.1170,-0.037105
5230323,2021-06-30 09:30:00,688981,55.19,55.19,54.49,54.99,54.795800,121147.00,-0.1634,1.1057,...,59.10,60.08,58.70,59.25,59.354400,291408.81,0.3217,2.6597,2.6597,-0.055402


In [12]:
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
time_list = [20150101,20150301,20150601,20150901,
             20160101,20160301,20160601,20160901,
             20170101,20170301,20170601,20170901,
             20180101,20180301,20180601,20180901,
             20190101,20190301,20190601,20190901,
             20200101,20200301,20200601,20200901,
             20210101,20210301,20210601]

def save(start_time,end_time):
    final_df[
        (final_df['timestamp'] >= start_time) & 
        (final_df['timestamp'] < end_time)].to_parquet(
        path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par"
    )
    print("Save to ",path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par")
    return None

pool = mp.Pool(10)
t1 = time.time()
for i in tqdm(range(len(time_list)-1)):
    start_time = pd.to_datetime(str(time_list[i]))
    end_time = pd.to_datetime(str(time_list[i+1]))
    pool.apply_async(save, args=(start_time,end_time))  #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 3751.48it/s]


Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Data/2015-01-01_2015-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Data/2017-01-01_2017-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Data/2016-01-01_2016-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Data/2017-03-01_2017-06-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Data/2015-03-01_2015-06-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Data/2016-06-01_2016-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Data/2016-03-01_2016-06-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Data/2015-06-01_2015-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Data/2017-06-01_2017-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Data/2016-09-01_2017-

# Get Factor

In [14]:
day = 30
stride = 5
kernal_size = 7
def generateC(l1):
    if len(l1) == 1:
        return []
    v = [[l1[0], i] for i in l1[1:]]
    l1 = l1[1:]
    return v + generateC(l1)

def generate_Num_and_ReversedNum(feat_nums):
    list1 = list(range(feat_nums))
    num = generateC(list1)
    num_rev = []
    for l in num:
        l1 = l.copy()
        l1.reverse()
        num_rev.append(l1)
    return num, num_rev

def generate_Step_List(stride,data_length = 30,kernal_size = None):
    """
    data_length: days
    kernal_size: how many days'moving
    stride
    """
    step_list = []
    start , end = 0 , 0
    if kernal_size == None:
        kernal_size = stride
    while end < data_length:
        end = start + kernal_size
        if end >= data_length:
            end = data_length
            start = end - kernal_size
        step_list.append([start,end])
        start += stride
    return step_list

In [15]:
generate_Step_List(stride = stride,data_length=30,kernal_size=kernal_size)

[[0, 7], [5, 12], [10, 17], [15, 22], [20, 27], [23, 30]]

In [3]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 36253.96it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [17]:
# Read Data

def cov(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        mean1 = sub_data1.mean(axis=4, keepdims=True)  # (2000, 1, 36, 2, 1)
        mean2 = sub_data2.mean(axis=4, keepdims=True)
        spread1 = sub_data1 - mean1  # (2000, 1, 36, 2, 10)
        spread2 = sub_data2 - mean2
        cov = ((spread1 * spread2).sum(axis=4, keepdims=True) / (sub_data1.shape[4] - 1)).mean(axis=3,
                                                                                                keepdims=True)  # (2000, 1, 36, 1, 1)
        l.append(cov)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def corr(x,num,num_rev,step_list,output_path):
    """Main Function"""
    l = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        std1 = sub_data1.std(axis=4, keepdims=True)
        std2 = sub_data2.std(axis=4, keepdims=True)
        std = (std1 * std2).mean(axis=3, keepdims=True)
        l.append(std)
    l = np.squeeze(np.array(l)).transpose(1, 2, 0)
    fct = (sub_data1.shape[4] - 1) / sub_data1.shape[4]
    
    # read cov
    cov = pd.read_parquet(output_path.replace("Corr","Cov"))
    cov = np.array(cov.set_index(['timestamp','ticker']))
    cov = cov.reshape(l.shape)
    # divide
    l = np.divide(cov, l, out=np.zeros_like(cov, dtype=np.float64), where=l!=0) * fct
    return l

def std(x,num,num_rev,step_list):
    """Main Function"""
    std = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        sub_data1 = x[:, :, :, start:end]
        std1 = sub_data1.std(axis=3, keepdims=True)
        std.append(std1)
    return np.squeeze(np.array(std)).transpose(1, 2, 0)

def zscore(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        sub_data1 = x[:, :, :, start:end]
        mean = sub_data1.mean(axis=3, keepdims=True)
        std = sub_data1.std(axis=3, keepdims=True)
        z_score = np.divide(mean, std, out=np.zeros_like(mean, dtype=np.float64), where=std!=0)
        l.append(z_score)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def ret(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        sub_data1 = x[:, :, :, start:end]
        ret = np.divide(sub_data1[:, :, :, -1], sub_data1[:, :, :, 0], out=sub_data1[:, :, :, -1], where=sub_data1[:, :, :, 0]!=0) - 1
        l.append(ret)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def decay(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(step_list):
        start = i[0]
        end = i[1]
        time_spread = end - start
        weight = np.arange(1, time_spread + 1)
        weight = weight / (weight.sum())
        sub_data1 = (x[:, :, :, start:end] * weight).sum(axis=3, keepdims=True)
        l.append(sub_data1)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def test(input_path,name,output_path,method = None):
    dataframe_list = pd.read_parquet(input_path + name)
    dataframe_list = dataframe_list.set_index(["timestamp","ticker"])
    x = np.array(dataframe_list.drop('target',axis=1))
    x = x.reshape(x.shape[0], 1, day, -1)
    x = x.transpose(0, 1, 3, 2)

    feat_nums = x.shape[2]
    data_length = x.shape[3]
    num, num_rev = generate_Num_and_ReversedNum(feat_nums)
    step_list = generate_Step_List(stride = stride,data_length=data_length,kernal_size=kernal_size)

    """Main Function"""
    if method == "Cov":
        l = cov(x,num,num_rev,step_list)
    if method == "Corr":
        l = corr(x,num,num_rev,step_list,output_path+name)
    if method == "Std":
        l = std(x,num,num_rev,step_list)
    if method == "Zscore":
        l = zscore(x,num,num_rev,step_list)
    if method == "Return":
        l = ret(x,num,num_rev,step_list)
    if method == "Decay":
        l = decay(x,num,num_rev,step_list)

    l = pd.DataFrame(l.reshape(l.shape[0],-1))
    l.index = dataframe_list.index
    l.columns = [str(i) for i in range(l.shape[1])]
    l.reset_index(inplace=True)
    l.to_parquet(output_path + name)
    print("save to ",output_path + name)
    return None

In [18]:
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Cov','Std','Zscore','Return','Decay']
for i in dir_list:
    for alpha in path_list:
        output = path + i + "/" 
        pool.apply_async(test, args=(data_path,alpha,output,i))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 6/6 [00:01<00:00,  4.15it/s]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Cov/2015-01-01_2015-03-01.par


100%|██████████| 6/6 [00:15<00:00,  2.60s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Cov/2016-01-01_2016-03-01.par


 67%|██████▋   | 4/6 [00:17<00:08,  4.33s/it]

...

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Decay/2021-01-01_2021-03-01.par


 67%|██████▋   | 4/6 [00:00<00:00,  7.69it/s]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Decay/2020-06-01_2020-09-01.par


100%|██████████| 6/6 [00:00<00:00,  9.01it/s]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Decay/2021-03-01_2021-06-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Decay/2020-09-01_2021-01-01.par
并行执行时间： 154


In [19]:
t1 = time.time()
pool = mp.Pool(10)
for alpha in path_list:
    output = path + "Corr" + "/" 
#     test(data_path,alpha,output,"Corr")
    pool.apply_async(test, args=(data_path,alpha,output,"Corr"))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

  0%|          | 0/6 [00:00<?, ?it/s]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Corr/2015-01-01_2015-03-01.par


 67%|██████▋   | 4/6 [00:13<00:06,  3.46s/it]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Corr/2016-01-01_2016-03-01.par


 67%|██████▋   | 4/6 [00:15<00:08,  4.08s/it]

...

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Corr/2021-01-01_2021-03-01.par


 83%|████████▎ | 5/6 [00:24<00:04,  4.94s/it]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Corr/2020-06-01_2020-09-01.par


100%|██████████| 6/6 [00:36<00:00,  6.02s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Corr/2021-03-01_2021-06-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Corr/2020-09-01_2021-01-01.par
并行执行时间： 113


# Feat Concat ( Save to Feat_Concat file)

In [32]:
os.mkdir(path+"Feat_Concat")

In [3]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 32417.33it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [21]:
def test(alpha,dir_list):
    feat_cat = []
    for i in dir_list:
        feat_cat.append(pd.read_parquet(path + i + "/" + alpha).set_index(['timestamp','ticker']))
    feat_cat = pd.concat(feat_cat,axis=1)
    feat_cat.columns = [str(i) for i in range(feat_cat.shape[1])]
    feat_cat.reset_index().to_parquet(path+"Feat_Concat/"+alpha)
    print(path+"Feat_Concat/"+alpha)
    return None

In [22]:
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Cov','Corr','Std','Zscore','Return','Decay']
final = []
for alpha in tqdm(path_list):
#     test(alpha,dir_list)
    pool.apply_async(test, args=(alpha,dir_list))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 64148.18it/s]


/home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Feat_Concat/2015-01-01_2015-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Feat_Concat/2017-01-01_2017-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Feat_Concat/2016-01-01_2016-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Feat_Concat/2018-01-01_2018-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Feat_Concat/2017-03-01_2017-06-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Feat_Concat/2015-03-01_2015-06-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Feat_Concat/2015-06-01_2015-09-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Feat_Concat/2016-03-01_2016-06-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Feat_Concat/2016-06-01_2016-09-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Feat_Concat/2015-09-01_2016-01-01.par
/home/wuwe

In [4]:
pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_10d_return_move7/Feat_Concat/2015-03-01_2015-06-01.par")

,timestamp,ticker,0,1,2,3,4,5,6,7,...,638,639,640,641,642,643,644,645,646,647
0,2015-03-02 09:30:00,1,1356.684188,84.211645,305.934957,345.439695,157.255464,32.632962,1487.980910,277.481167,...,0.999382,1.038779,0.727014,0.793861,3.094361,2.806043,1.981571,2.059679,1.441482,1.574039
1,2015-03-03 09:30:00,1,1514.645083,132.414417,185.943348,320.944007,102.432443,2.643398,1821.394529,297.463117,...,0.938532,0.919450,0.687725,0.860550,3.348075,2.662504,1.860929,1.823046,1.363564,1.706282
2,2015-03-04 09:30:00,1,1547.077226,148.908424,194.353805,323.140950,39.377121,5.164912,1787.884374,249.472157,...,1.167886,0.825050,0.775739,0.922700,3.228461,2.470561,2.315682,1.635886,1.538075,1.829504
3,2015-03-05 09:30:00,1,1432.151831,244.562864,218.119383,336.025436,32.632962,96.370376,1643.154864,238.639079,...,1.151889,0.811546,0.793861,0.916464,3.180207,2.269082,2.283971,1.609096,1.574039,1.817136
4,2015-03-06 09:30:00,1,1153.484800,400.814012,319.736495,122.652576,2.643398,240.646655,1415.693550,344.222212,...,1.122236,0.752807,0.860550,0.910300,2.915457,2.105493,2.225161,1.492614,1.706282,1.804893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165502,2015-05-25 09:30:00,603998,2.240274,1.463490,0.941517,6.271476,12.625167,7.032436,2.404124,2.528236,...,6.254932,10.096500,10.915346,9.235907,9.053157,7.218514,6.254932,10.096500,10.915346,9.235907
165503,2015-05-26 09:30:00,603998,3.911567,1.426376,1.217305,10.111400,12.001419,7.462433,3.419871,2.453850,...,5.706407,10.369582,9.786796,9.790382,7.787754,7.244864,5.706407,10.369582,9.786796,9.790382
165504,2015-05-27 09:30:00,603998,3.276300,1.792205,1.812945,11.865157,9.035579,23.512514,3.126076,2.650729,...,7.431554,10.883589,9.457114,10.488761,7.984779,7.784643,7.431554,10.883589,9.457114,10.488761
165505,2015-05-28 09:30:00,603998,1.714350,0.550348,1.657655,9.549179,7.032436,41.609586,2.186367,1.247900,...,7.926150,10.518246,9.235907,10.388107,7.898525,7.845754,7.926150,10.518246,9.235907,10.388107


# Reshape

In [24]:
def test(i):
    a = pd.read_parquet(path + "Feat_Concat/" + i).set_index(["timestamp","ticker"])
    index = a.index
    a = np.array(a).reshape(np.array(a).shape[0],-1,5)
    a = a.transpose(0,2,1)
    a = a.reshape(a.shape[0],-1)
    a = pd.DataFrame(a)
    a.index = index
    a.columns = [str(i) for i in a.columns]
    a.to_parquet(path + "Feat_Concat/" + i)
    print ("Save to " + path + "Feat_Concat/" + i)
    return None

In [25]:
pool = mp.Pool(10)
t1 = time.time()
for alpha in tqdm(path_list):
#     test(alpha)
    pool.apply_async(test, args=(alpha,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 63960.06it/s]


并行执行时间： 19


# Target

In [4]:
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
configs = namespace.load_namespace(r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template')
FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/feature_platform")
FT.load_feature_from_file(["10d_ret_standscalar"], "2015-01-01", "2022-01-01", universe='Investable', timedelta=None)
a = FT.feature_data
del FT
a.columns = ["target"]
a.reset_index(inplace=True)
a["timestamp"] = pd.to_datetime(a["timestamp"])
a["ticker"] = a["ticker"].apply(lambda x: str(x).zfill(6))
a.set_index(["timestamp","ticker"],inplace=True)
a

Loading Feature...: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1705/1705 [00:03<00:00, 536.94it/s]


target
timestamp           ticker           
2015-01-12 09:30:00 000627  10.497501
                    002095   6.362775
                    000626   6.354776
                    600288   6.353383
                    600986   4.734728
...                               ...
2021-07-01 09:30:00 603355  -2.436583
                    688356  -2.958048
                    600093  -3.061266
                    688277  -3.216929
                    688626  -3.502215

[5067767 rows x 1 columns]

Process ForkPoolWorker-35:
Process ForkPoolWorker-12:
Process ForkPoolWorker-59:
Process ForkPoolWorker-56:
Process ForkPoolWorker-18:
Process ForkPoolWorker-61:
Process ForkPoolWorker-66:
Process ForkPoolWorker-53:
Process ForkPoolWorker-36:
Process ForkPoolWorker-30:
Process ForkPoolWorker-25:
Process ForkPoolWorker-24:
Process ForkPoolWorker-23:
Process ForkPoolWorker-55:
Process ForkPoolWorker-48:
Process ForkPoolWorker-51:
Process ForkPoolWorker-57:
Process ForkPoolWorker-68:
Process ForkPoolWorker-64:
Process ForkPoolWorker-52:
Process ForkPoolWorker-11:
Process ForkPoolWorker-15:
Process ForkPoolWorker-3:
Process ForkPoolWorker-72:
Process ForkPoolWorker-10:
Process ForkPoolWorker-22:
Process ForkPoolWorker-32:
Process ForkPoolWorker-17:
Process ForkPoolWorker-58:
Process ForkPoolWorker-70:
Process ForkPoolWorker-65:
Process ForkPoolWorker-31:
Process ForkPoolWorker-2:
Process ForkPoolWorker-9:
Process ForkPoolWorker-63:
Process ForkPoolWorker-69:
Process ForkPoolWorker-8:
Proce

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-40:
Process ForkPoolWorker-42:
Process ForkPoolWorker-41:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-45:
Proc

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
  

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/minicond

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/mul

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local

In [5]:
def test(j):
    return pd.read_parquet(j)
pool = mp.Pool(10)
result = {}
t1 = time.time()
for i in tqdm(path_list):
    result[i] = (pool.apply_async(test, args=(path+"Feat_Concat/"+i,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))
df_list = []
for i in tqdm(result):
    df_list.append(result[i].get())
final_df = pd.concat(df_list,axis=0)
final_df["timestamp"] = pd.to_datetime(final_df["timestamp"])
final_df["ticker"] = final_df["ticker"].apply(lambda x: str(x).zfill(6))
final_df.set_index(["timestamp","ticker"],inplace=True)
final_df

100%|██████████| 26/26 [00:00<00:00, 79252.84it/s]

并行执行时间： 85


,,0,1,2,3,4,5,6,7,8,9,...,638,639,640,641,642,643,644,645,646,647
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,000001,1278.352474,1356.684188,84.211645,305.934957,345.439695,336.025436,1139.086324,1487.980910,277.481167,240.718855,...,1.415200,0.999382,1.038779,0.811546,3.275843,3.094361,2.806043,1.981571,2.059679,1.609096
2015-02-17 09:30:00,000001,1030.443350,1514.645083,132.414417,185.943348,320.944007,122.652576,839.606114,1821.394529,297.463117,90.108840,...,1.342818,0.938532,0.919450,0.752807,2.975504,3.348075,2.662504,1.860929,1.823046,1.492614
2015-02-25 09:30:00,000001,463.879957,1547.077226,148.908424,194.353805,323.140950,157.255464,406.156588,1787.884374,249.472157,61.761414,...,1.246000,1.167886,0.825050,0.727014,2.779632,3.228461,2.470561,2.315682,1.635886,1.441482
2015-02-26 09:30:00,000001,333.462852,1432.151831,244.562864,218.119383,336.025436,102.432443,294.482502,1643.154864,238.639079,109.499467,...,1.144404,1.151889,0.811546,0.687725,2.808371,3.180207,2.269082,2.283971,1.609096,1.363564
2015-02-27 09:30:00,000001,725.282183,1153.484800,400.814012,319.736495,122.652576,39.377121,810.496650,1415.693550,344.222212,228.711740,...,1.061889,1.122236,0.752807,0.775739,3.140421,2.915457,2.105493,2.225161,1.492614,1.538075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-25 09:30:00,688981,0.469733,0.431438,0.112919,0.530643,0.806602,0.647112,0.338650,0.307002,0.137788,0.581955,...,1.479964,1.470393,1.545489,1.289929,1.655479,1.657732,1.479964,1.470393,1.545489,1.289929
2021-05-26 09:30:00,688981,0.084924,0.408133,0.160417,0.320429,0.497645,0.214633,0.126312,0.367419,0.152883,0.353538,...,1.370139,1.595339,1.465582,1.406275,1.724904,1.750896,1.370139,1.595339,1.465582,1.406275
2021-05-27 09:30:00,688981,0.072833,0.526481,0.202421,0.891798,0.535207,0.158688,0.066133,0.499931,0.160231,0.855236,...,1.400039,1.819846,1.293896,1.474754,1.789696,1.643843,1.400039,1.819846,1.293896,1.474754


In [6]:
final_df = pd.merge(final_df,a,how="inner",left_index=True,right_index=True)
final_df

,,0,1,2,3,4,5,6,7,8,9,...,639,640,641,642,643,644,645,646,647,target
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,000001,1278.352474,1356.684188,84.211645,305.934957,345.439695,336.025436,1139.086324,1487.980910,277.481167,240.718855,...,0.999382,1.038779,0.811546,3.275843,3.094361,2.806043,1.981571,2.059679,1.609096,-0.940003
2015-02-17 09:30:00,000001,1030.443350,1514.645083,132.414417,185.943348,320.944007,122.652576,839.606114,1821.394529,297.463117,90.108840,...,0.938532,0.919450,0.752807,2.975504,3.348075,2.662504,1.860929,1.823046,1.492614,-0.700700
2015-02-25 09:30:00,000001,463.879957,1547.077226,148.908424,194.353805,323.140950,157.255464,406.156588,1787.884374,249.472157,61.761414,...,1.167886,0.825050,0.727014,2.779632,3.228461,2.470561,2.315682,1.635886,1.441482,-0.605842
2015-02-26 09:30:00,000001,333.462852,1432.151831,244.562864,218.119383,336.025436,102.432443,294.482502,1643.154864,238.639079,109.499467,...,1.151889,0.811546,0.687725,2.808371,3.180207,2.269082,2.283971,1.609096,1.363564,-0.363216
2015-02-27 09:30:00,000001,725.282183,1153.484800,400.814012,319.736495,122.652576,39.377121,810.496650,1415.693550,344.222212,228.711740,...,1.122236,0.752807,0.775739,3.140421,2.915457,2.105493,2.225161,1.492614,1.538075,0.048317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-25 09:30:00,688981,0.469733,0.431438,0.112919,0.530643,0.806602,0.647112,0.338650,0.307002,0.137788,0.581955,...,1.470393,1.545489,1.289929,1.655479,1.657732,1.479964,1.470393,1.545489,1.289929,-0.257516
2021-05-26 09:30:00,688981,0.084924,0.408133,0.160417,0.320429,0.497645,0.214633,0.126312,0.367419,0.152883,0.353538,...,1.595339,1.465582,1.406275,1.724904,1.750896,1.370139,1.595339,1.465582,1.406275,-0.220639
2021-05-27 09:30:00,688981,0.072833,0.526481,0.202421,0.891798,0.535207,0.158688,0.066133,0.499931,0.160231,0.855236,...,1.819846,1.293896,1.474754,1.789696,1.643843,1.400039,1.819846,1.293896,1.474754,-0.209516


In [10]:
def a(df: pd.DataFrame, output_name: str, output_path: str):
    pool = mp.Pool(10)
    t1 = time.time()
    for date, group in tqdm(df.groupby('timestamp')):
        pool.apply_async(__a, args=(date,group))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
    pool.close()
    pool.join()
    t2 = time.time()
    print("并行执行时间：", int(t2 - t1))
def __a(date,group):
    date_format = pd.to_datetime(date).date()
    assert DataAPI.is_trading_day(date), f"{date} is not a trading date!"
    file_name = datetime.date.strftime(date_format, '%Y%m%d') + '.par'
    folder = os.path.join(output_path, output_name, str(date_format.year))
    if not os.path.exists(folder):
        os.makedirs(folder)
    file = os.path.join(folder, file_name)
    group.to_parquet(file)
    return None

In [ ]:
a(df = final_df, output_name = "Original_Input_Moving_7_Shift_5", 
                                   output_path= "/home/ShareFolder/feature_platform/ti0/wuwenjun/#Data_lib")

  0%|          | 0/1529 [00:00<?, ?it/s]Process ForkPoolWorker-260:
Process ForkPoolWorker-262:
Process ForkPoolWorker-259:
Process ForkPoolWorker-261:
Process ForkPoolWorker-254:
Process ForkPoolWorker-258:
Process ForkPoolWorker-256:
Process ForkPoolWorker-255:
Process ForkPoolWorker-257:
Process ForkPoolWorker-253:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/local/miniconda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 412, in _handle_workers
    pool._maintain_pool()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 248, in _maintain_pool
    self._repopulate_pool()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 241, in _repopulate_pool
    w.start()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 112, in start
    self._popen = self._Popen(self)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/context.py", line 277, in _Popen
    return Popen(process_obj)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/popen_fork.py", line 20, in __init__
   

In [9]:
del final_df

# Pooling ( Save to Pooling)

In [18]:
os.mkdir(path+"Pooling")

In [19]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 229100.64it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [21]:
""" Multi Processing"""
pool = mp.Pool(10)
t1 = time.time()
def test(file):
    array = pd.read_parquet(path + "Feat_Concat/" + file).set_index(["timestamp","ticker"])
    index = array.index
    array = np.array(array)
    array = torch.from_numpy(array.reshape(array.shape[0],-1,3))
    # Pooling
    max_pooling = torch.nn.MaxPool1d(kernel_size = 3, stride=1)(array) # Max Pooling
    avg_pooling = torch.nn.AvgPool1d(kernel_size = 3, stride=1)(array) # Mean Pooling
    min_pooling = -(torch.nn.MaxPool1d(kernel_size = 3, stride=1)(-array)) # MinPooling
    array = torch.cat([max_pooling,avg_pooling,min_pooling], axis=2)
    array = array.flatten(start_dim=1)
    array = pd.DataFrame(array.detach().numpy())
    array.index = index
    array.columns = [str(i) for i in range(array.shape[1])]
    array.reset_index(inplace=True)
#     # Daily Norm
#     final = []
#     for day in tqdm(array['timestamp'].drop_duplicates()):
#         df = array[array['timestamp'] == day].set_index(['timestamp','ticker'])
#         index = df.index
#         # Batch
#         df = torch.from_numpy(np.array(df)).to(torch.float) 
#         batch = nn.BatchNorm1d(df.shape[1])
#         df = batch(df)
#         df = pd.DataFrame(df.detach().numpy())
#         df.index = index
#         final.append(df)

#     final = pd.concat(final,axis=0)
#     final.reset_index(inplace=True)
    array.to_parquet(path + "Pooling/" + file)
    print("Save to : ", path + "Pooling/" + file)
    return None
for file in path_list:
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2015-01-01_2015-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2016-01-01_2016-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2017-01-01_2017-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2015-03-01_2015-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2016-06-01_2016-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2015-06-01_2015-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2017-03-01_2017-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2016-03-01_2016-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2017-06-01_2017-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Pooling/2018-01-01_2018-03-01.par
Save to : 

# Normalize Target and Concat

In [45]:
if os.path.exists(path+"Final") == False:
    os.makedirs(path+"Final")

In [46]:
dataframe_list = pd.read_csv(path + "data.csv").set_index(["timestamp","ticker"])
df = dataframe_list[target].unstack()
df = (df.T - df.mean(axis=1)) / df.std(axis=1)
df = df.T.stack()
df = pd.DataFrame(df,columns=["target"])
# to date time
df.reset_index(inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index(["timestamp","ticker"],inplace=True)
df

target
timestamp           ticker          
2015-01-06 09:30:00 1      -1.126526
                    2      -1.411284
                    4       0.122665
                    5       0.042938
                    6      -0.898595
...                              ...
2021-07-01 09:30:00 688700  2.186672
                    688777  0.631149
                    688788 -1.974877
                    688819  0.325758
                    688981 -0.745459

[5357704 rows x 1 columns]

In [47]:
def test(i):
    dataframe_list = pd.read_parquet(path +"Feat_Concat/" + i).set_index(["timestamp","ticker"])
    dataframe_list = pd.merge(dataframe_list,df,how="left",left_index=True,right_index=True)
    dataframe_list.reset_index().to_parquet(path +"Final/" + i)
    print("Save to : ",path +"Final/" + i)
    return None

In [48]:
pool = mp.Pool(10)
t1 = time.time()
for file in tqdm(path_list):
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()

t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 27256.16it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2015-01-01_2015-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2016-01-01_2016-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2017-01-01_2017-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2015-03-01_2015-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2015-06-01_2015-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2018-01-01_2018-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2016-09-01_2017-01-01.par
Save to :  Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2016-03-01_2016-06-01.par/home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2017-06-01_2017-09-01.par

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2016-06-01_2016-09-01.par
Save to :  /home/wuwenjun/Data

In [49]:
pd.read_parquet(path + "Final/" + "2019-06-01_2019-09-01.par")

,timestamp,ticker,0,1,2,3,4,5,6,7,...,315,316,317,318,319,320,321,322,323,target
0,2019-06-03 09:30:00,1,642.668930,732.445878,90.182527,506.161342,842.470287,127.050138,487.486112,560.657678,...,-0.511487,-0.417024,-0.306245,0.675007,0.596138,0.417038,1.522942,1.344993,0.940935,0.843771
1,2019-06-04 09:30:00,1,2096.386494,548.990224,109.882809,2174.762801,702.839116,171.892827,2208.559264,477.148118,...,-1.709293,-0.320815,-0.686616,0.779296,0.557267,0.501880,1.758233,1.257305,1.132351,0.878615
2,2019-06-05 09:30:00,1,3675.248588,387.817611,329.337998,3527.494588,478.741251,408.572213,3488.742740,237.657136,...,-1.407602,0.007567,-0.692376,0.762180,0.526562,0.522049,1.719625,1.188007,1.177871,0.599032
3,2019-06-06 09:30:00,1,5095.576744,338.420328,330.896680,4833.440429,433.775567,347.356847,4724.327142,211.071372,...,-1.722898,-0.175167,-0.403533,0.728784,0.475635,0.515751,1.644287,1.073122,1.163647,0.392347
4,2019-06-10 09:30:00,1,6144.335576,450.831178,404.073763,6296.420919,502.373333,381.229130,6353.789056,299.574733,...,-2.083133,-0.313553,-0.416336,0.759853,0.446695,0.473500,1.714376,1.007829,1.068298,0.334024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231011,2019-08-26 09:30:00,603999,0.086600,0.485252,4.737556,0.063339,0.418676,4.186172,0.072861,0.462228,...,1.540904,-1.161398,3.116449,0.682538,0.835436,4.423025,1.618849,1.981433,10.490544,-1.556930
231012,2019-08-27 09:30:00,603999,0.227422,0.383404,4.298236,0.137667,0.373142,3.818259,0.184933,0.368693,...,1.497273,-0.786195,1.630575,1.075733,0.732858,4.645142,2.551415,1.738171,11.017364,-1.121929
231013,2019-08-28 09:30:00,603999,0.263048,0.336760,3.575583,0.180887,0.306350,3.306194,0.227586,0.303971,...,1.168284,-0.831340,1.129598,1.208607,0.640404,4.631107,2.866544,1.518891,10.984069,-0.865651
231014,2019-08-29 09:30:00,603999,0.291648,0.236471,2.697064,0.214412,0.201980,2.628958,0.249952,0.211949,...,0.589565,-0.419440,-0.111805,1.204451,0.573242,4.510796,2.856662,1.359589,10.698702,-0.440248
